<h1>datasets paths</h1>
<h3>https://www.kaggle.com/chrisfilo/urbansound8k</h3>
<h3>https://www.kaggle.com/mathurinache/the-lj-speech-dataset</h3>

In [1]:
import librosa
import librosa.display
import IPython as ip
import os
from tqdm import tqdm
import numpy as np
import pandas as pd
import os
import pickle

from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Conv2D, ReLU, BatchNormalization, Flatten, Dense, Reshape, Conv2DTranspose, Activation, Lambda
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError
import numpy as np
import tensorflow as tf

<h3>for this i will use a vae (variational auto encoder) and train it to get as input an audio containing a noise from the urbain dataset and then use the spectrogram of this noisy audio to generate a new spectrogram of the audio without the noise for this we need to get a voice speaking dataset and mixe it with the noise so how could we mix 2 audio files in librosa?</h3>

In [2]:
# here we read both audios and combine them in a one audio using librosa
audio1 = '../input/the-lj-speech-dataset/LJSpeech-1.1/wavs/LJ001-0004.wav'
audio2= '../input/urbansound8k/fold4/115415-9-0-6.wav'
# the audios must have the same durations
aud1, sample_rate1 = librosa.load(audio1, mono=True, duration=2, sr=16384)
aud2, sample_rate2 = librosa.load(audio2, mono=True, duration=2, sr=16384)

# MERGE
combined = (aud1+aud2)/2
combined_sample_rate = int((sample_rate1+sample_rate2)/2)

# DISPLAY
print("the mixed audio")
librosa.display.waveplot(combined, sr=combined_sample_rate)
ip.display.Audio(combined, rate=combined_sample_rate)

In [3]:
print("the original audio")
librosa.display.waveplot(aud1, sr=sample_rate1)
ip.display.Audio(aud1, rate=sample_rate1)

In [4]:
combined.shape

<h1>now lets start preparing the data</h1>
<h3>we will take 10000 human voice and mix it with 1 random sound from the 10 folders in urbain sounds so we will have around 10000 sample</h3>

In [5]:
# the nice thing with librosa is that we don't need to normelize it does it for us
# def normalise(array):
#     norm_array = (array - array.min()) / (array.max() - array.min())
#     return norm_array, (array.min(), array.max())

# def denormalise(array, original_min, original_max):
#     array = array * (original_max - original_min) + original_min
#     return array

In [6]:
# here i tried with log spectrogram but it didn't really work well
# def extract_log_spectrograms(human_wave_path, urban_wave_path):
#     n_fft = 512
#     hop_length = 256
#     aud1, sample_rate1 = librosa.load(human_wave_path, mono=True, duration=0.74)
#     aud2, sample_rate2 = librosa.load(urban_wave_path, mono=True, duration=0.74)
    
#     # MERGE
#     combined = (aud1+aud2)/2
#     combined_sample_rate = int((sample_rate1+sample_rate2)/2)
    
#     # extract log spectrogram
#     stft = librosa.stft(combined, n_fft=n_fft, hop_length=hop_length)
#     spectrogram = np.abs(stft)
#     x_log_spectrogram = librosa.amplitude_to_db(spectrogram)
#     x_log_spectrogram, min_max = normalise(x_log_spectrogram)
    
#     stft = librosa.stft(aud1, n_fft=n_fft, hop_length=hop_length)
#     spectrogram = np.abs(stft)
#     y_log_spectrogram = librosa.amplitude_to_db(spectrogram)
#     y_log_spectrogram, _ = normalise(y_log_spectrogram)
        
#     return x_log_spectrogram, y_log_spectrogram, min_max

In [7]:
def extract_wave(human_wave_path, urban_wave_path):
    aud1, sample_rate1 = librosa.load(human_wave_path, mono=True, duration=2, sr=16384)
    aud2, sample_rate2 = librosa.load(urban_wave_path, mono=True, duration=2, sr=16384)
    
    # MERGE
    combined = (aud1+aud2)/2
    combined_sample_rate = int((sample_rate1+sample_rate2)/2)
        
    return combined, aud1, combined_sample_rate

In [8]:
# here we are going to use only the aire conditionning sound
urban_csv = pd.read_csv("../input/urbansound8k/UrbanSound8K.csv")
urban_csv = urban_csv[urban_csv["class"] == "air_conditioner"]
urban_csv = urban_csv[urban_csv["fold"] == 5]
urban_csv

In [9]:
air_conditionning_folders = urban_csv["slice_file_name"]

In [10]:
x_train, y_train = [],[]
def data_pipeline(human_voices_folder,urban_sounds):
    human_voice_waves = os.listdir(human_voices_folder)
    for i in tqdm(range(8500)):
        human_voice_wave = human_voice_waves[i]
        rand_sampled_urban = np.random.choice(air_conditionning_folders)
        try:
            urban_path = os.path.join(str(urban_sounds),str(rand_sampled_urban))
            human_path = os.path.join(str(human_voices_folder),str(human_voice_wave))
            x, y, _ = extract_wave(human_path, urban_path)
            x_train.append(x)
            y_train.append(y)
        except:
            pass
                
    return np.array(x_train), np.array(y_train)

In [11]:
human_voices_folder= "../input/the-lj-speech-dataset/LJSpeech-1.1/wavs"
urban_sounds = "../input/urbansound8k/fold5"
x, y = data_pipeline(human_voices_folder,urban_sounds)

In [12]:
x.shape, y.shape

In [13]:
# we reshape for the un conv layer
x = x.reshape(-1,x.shape[1], 1)
y = y.reshape(-1,y.shape[1], 1)

<h1>the autoencoder</h1>

In [77]:
model = tf.keras.Sequential()

model.add(tf.keras.layers.Conv1D(filters=32, strides=32, input_shape=(16384*2,1), padding="same", kernel_size=2048, use_bias=True))
# model.add(tf.keras.layers.Conv1D(filters=64, strides=64, padding="same", kernel_size=2048, use_bias=True))
model.add(Activation("tanh"))
# model.add(tf.keras.layers.Conv1DTranspose(filters=32, strides=64, padding="same", kernel_size=2048, use_bias=True))
model.add(tf.keras.layers.Conv1DTranspose(filters=1, strides=32, padding="same", kernel_size=2048, use_bias=True))
model.add(Activation("tanh"))

model.compile(optimizer="adam",loss="mse")

In [78]:
model.summary()

In [79]:
model.fit(x, y, epochs=100)

In [89]:
# lets try it
human_wave_path = "../input/the-lj-speech-dataset/LJSpeech-1.1/wavs/LJ001-0005.wav"
urban_wave_path = "../input/urbansound8k/fold5/100852-0-0-6.wav"
x_wave, y_wave, sr = extract_wave(human_wave_path, urban_wave_path)

In [90]:
x_wave = x_wave.reshape(1,x.shape[1],1)
x_wave.shape

In [91]:
y_pred = model.predict(x_wave)

In [92]:
y_pred = y_pred.reshape(x.shape[1])

In [93]:
librosa.display.waveplot(y_pred , sr=16384)

In [94]:
librosa.display.waveplot(y_wave , sr=16384)

In [95]:
librosa.display.waveplot(x_wave.reshape(x.shape[1]) , sr=16384)

In [96]:
ip.display.Audio(y_pred , rate=16384)

In [97]:
ip.display.Audio(x_wave.reshape(x.shape[1]) , rate=16384)

In [ ]:
# actually its working good the speech is more clear